# EMLOpt
Master Thesis in Artificial Intelligence at University of Bologna, a.y. 2021/2022

Daniele Verì, Michele Lombardi, Andrea Borghesi, Stefano Teso


In [ ]:
!unzip eml-opt.zip
!pip install -r eml-opt/dependencies/requirements.txt
!git clone https://github.com/DanieleVeri/emllib.git eml-opt/dependencies/emllib
!mkdir eml-opt/dependencies/cplex
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/69b1b89a73a7d0931fbfdb355eb147c3 -O eml-opt/dependencies/cplex/cplex_studio1210.linux-x86-64.bin
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/97133b747b0114a4e3dba77ab26d68d5 -O eml-opt/dependencies/cplex/response.properties
!pip install docplex
!sh eml-opt/dependencies/cplex/cplex_studio1210.linux-x86-64.bin -f response.properties
!python3 /opt/ibm/ILOG/CPLEX_Studio1210/python/setup.py install

In [ ]:
import sys
sys.path.append("./eml-opt") 

%load_ext autoreload
%autoreload 2

In [ ]:
from emlopt.config import DEFAULT
from emlopt.search_loop import SearchLoop
from emlopt.problem import ConvexRealProblem
from emlopt.wandb import WandbExperiment

from problems.ackley import build_ackley, constraint_scbo


problem = ConvexRealProblem("test", *build_ackley(10), constraint_scbo)
search = SearchLoop(problem, DEFAULT)

wandb_cfg = WandbExperiment.get_defatult_cfg()
wandb_cfg['experiment_name'] = "navajo joe"

with WandbExperiment(wandb_cfg, search):
    search.run()
